In [124]:
from pathlib import Path
from sqlalchemy import create_engine

import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

# Se define la ruta para la base de datos local SQLite
project_name = Path.cwd().parent.parent.name
data_projetc_path = Path.home().joinpath(f'Dropbox/La Casa Del Carpintero/Departamento de Programación/data_projects_git/data_{project_name}')
db_file = 'comisiones.db'
db_file_path_str = str(data_projetc_path.joinpath(db_file))

# Se crea el objeto engine para trabajarlo con los metodos de pandas
engine = create_engine(f'sqlite:///{db_file_path_str}')

In [125]:
file01 = 'clientes_regalos'
file01_path_str = str(data_projetc_path.joinpath(f'comisiones/{file01}.xlsx'))
clientes_regalos = pd.read_excel(file01_path_str, dtype_backend='numpy_nullable')

In [126]:
with engine.connect() as conn, conn.begin():

    cobranza_año = pd.read_sql_table('cobranza_año_2024', conn, dtype_backend='numpy_nullable')

    costo_ventas_1_2024 = pd.read_sql_table('costo_ventas_1_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_2_2024 = pd.read_sql_table('costo_ventas_2_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_3_2024 = pd.read_sql_table('costo_ventas_3_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_4_2024 = pd.read_sql_table('costo_ventas_4_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_5_2024 = pd.read_sql_table('costo_ventas_5_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_6_2024 = pd.read_sql_table('costo_ventas_6_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_7_2024 = pd.read_sql_table('costo_ventas_7_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_8_2024 = pd.read_sql_table('costo_ventas_8_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_9_2024 = pd.read_sql_table('costo_ventas_9_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_10_2024 = pd.read_sql_table('costo_ventas_10_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_11_2024 = pd.read_sql_table('costo_ventas_11_2024', conn, dtype_backend='numpy_nullable')
    # costo_ventas_12_2024 = pd.read_sql_table('costo_ventas_12_2024', conn, dtype_backend='numpy_nullable')
    
    datos_com_2023 = pd.read_sql_table('datos_com_2023', conn, dtype_backend='numpy_nullable')
    bonos_limites_comisiones = pd.read_sql_table('bonos_limites_comisiones', conn, dtype_backend='numpy_nullable')
    # bonos_limites_comisiones_2024 = pd.read_sql_table('bonos_limites_comisiones_2024', conn, dtype_backend='numpy_nullable')
    # bonos_limites_comisiones_2025 = pd.read_sql_table('bonos_limites_comisiones_2025', conn, dtype_backend='numpy_nullable')

engine.dispose()

costo_ventas_list = [
            costo_ventas_1_2024,
            costo_ventas_2_2024,
            costo_ventas_3_2024,
            costo_ventas_4_2024,
            costo_ventas_5_2024,
            costo_ventas_6_2024,
            costo_ventas_7_2024,
            costo_ventas_8_2024,
            costo_ventas_9_2024,
            costo_ventas_10_2024,
            costo_ventas_11_2024,
            # costo_ventas_12_2024,
        ]

costo_ventas_año = pd.concat(costo_ventas_list)

In [127]:
def gerentes_list_func() -> pd.DataFrame:

    gerentes_list =  pd.DataFrame(
                [
                    [206, 'Mariana Araceli Carvajal Flores', 'Piso'],
                    [208, 'Carlos Manuel Guzmán Rios', 'CE'],
                    [209, 'Rosario Martinez Zarate', 'Piso'],
                ]
                ,
                columns = 
                    [
                        'salesperson_id',
                        'salesperson_name',
                        'business_model',
                    ]
            )

    return gerentes_list

In [128]:
def comisiones_mes_2024_func(mes:int, costo_ventas_list:list, gerentes_list:pd.DataFrame, bonos_limites_comisiones:pd.DataFrame, clientes_regalos:pd.DataFrame) -> pd.DataFrame:

    df = costo_ventas_list[mes-1]
    mes_df = df['invoice_date'].dt.month.unique()[0]
    
    df.loc[:, 'mes'] = mes_df
    gerentes_list['mes'] = mes_df


    comisiones_2024_vend = (df.loc[
        ~df['salesperson_name'].isin(gerentes_list['salesperson_name'])
        ,
                [
                    'salesperson_id',
                    'salesperson_name',
                    'business_model',
                    'mes',
                    'price_subtotal',
                    'utilidad_partida_$_after_cc'
                ]
            ].groupby('salesperson_id')
            .agg({
                'salesperson_name':['first'], 
                'business_model':['first'], 
                'mes':['first'], 
                'price_subtotal':['sum'],
                'utilidad_partida_$_after_cc':['sum']
                })
            .set_axis(
                [
                    'salesperson_name',
                    'business_model',
                    'mes',
                    'facturacion',
                    'utilidad'
                ], axis='columns')
            .reset_index()
    )


    comisiones_2024_ger = gerentes_list.copy()

    comisiones_2024_ger_values = (df.loc[:,
                [
                    'sale_team_description',
                    'mes',
                    'price_subtotal',
                    'utilidad_partida_$_after_cc'
                ]
            ].groupby('sale_team_description')
            .agg({
                'mes':['first'], 
                'price_subtotal':['sum'],
                'utilidad_partida_$_after_cc':['sum']
                })
            .set_axis(
                [
                    'mes',
                    'facturacion',
                    'utilidad'
                ], axis='columns')
            .reset_index()
    )


    comisiones_2024_ger.loc[
            comisiones_2024_ger['salesperson_name'] == 'Carlos Manuel Guzmán Rios'
            , 
            ['facturacion', 'utilidad']
        ] = (comisiones_2024_ger_values[
                    comisiones_2024_ger_values['sale_team_description'].isin(['A1 CE', 'A2 CE'])
                ]
                .groupby('mes')
                .sum()
                [['facturacion', 'utilidad']]
                .values
            )

    comisiones_2024_ger.loc[
            comisiones_2024_ger['salesperson_name'] == 'Mariana Araceli Carvajal Flores'
            , 
            ['facturacion', 'utilidad']
        ] = comisiones_2024_ger_values[
                    comisiones_2024_ger_values['sale_team_description'] == 'A1 Piso'
                ][['facturacion', 'utilidad']].values

    comisiones_2024_ger.loc[
            comisiones_2024_ger['salesperson_name'] == 'Rosario Martinez Zarate'
            , 
            ['facturacion', 'utilidad']
        ] = comisiones_2024_ger_values[
                    comisiones_2024_ger_values['sale_team_description'] == 'A2 Piso'
                ][['facturacion', 'utilidad']].values


    comisiones_mes_2024_sin_regalos = pd.concat([comisiones_2024_ger, comisiones_2024_vend], ignore_index=True)


    clientes_regalos_mes = clientes_regalos.loc[
                                clientes_regalos['fecha_regalo'].dt.month == mes, 
                                    [
                                        'Sale team description', 
                                        'salesperson_id', 
                                        'costo_regalo_$'
                                    ]
                                ].groupby('salesperson_id').agg(
                                    {
                                        'Sale team description':'first',
                                        'costo_regalo_$':'sum',
                                    }
                                ).reset_index()
    
    
    clientes_regalos_mes_salesteam = clientes_regalos_mes[
                                            ['Sale team description', 'costo_regalo_$']
                                    ].groupby('Sale team description').sum().reset_index()


    comisiones_mes_2024 = comisiones_mes_2024_sin_regalos.merge(clientes_regalos_mes, how='left', on='salesperson_id')

    comisiones_mes_2024.loc[
            comisiones_mes_2024['salesperson_name'] == 'Carlos Manuel Guzmán Rios'
            , 
            'costo_regalo_$'
        ] = clientes_regalos_mes_salesteam.loc[
                clientes_regalos_mes_salesteam['Sale team description'].isin(['A1 CE', 'A2 CE'])
                ,
                'costo_regalo_$'
            ].sum()


    comisiones_mes_2024.loc[
            comisiones_mes_2024['salesperson_name'] == 'Mariana Araceli Carvajal Flores'
            , 
            'costo_regalo_$'
        ] = clientes_regalos_mes_salesteam.loc[
                clientes_regalos_mes_salesteam['Sale team description'].isin(['A1 Piso'])
                ,
                'costo_regalo_$'
            ].sum()
    
    comisiones_mes_2024.loc[
            comisiones_mes_2024['salesperson_name'] == 'Rosario Martinez Zarate'
            , 
            'costo_regalo_$'
        ] = clientes_regalos_mes_salesteam.loc[
                clientes_regalos_mes_salesteam['Sale team description'].isin(['A2 Piso'])
                ,
                'costo_regalo_$'
            ].sum()


    comisiones_mes_2024['utilidad_$_after_regalos'] = comisiones_mes_2024['utilidad'] - comisiones_mes_2024['costo_regalo_$']


    for i in range(len(comisiones_mes_2024)):

        if comisiones_mes_2024.iloc[i]['salesperson_name'] in gerentes_list['salesperson_name'].values:
            
            bonos_limites_comisiones_line = bonos_limites_comisiones.loc[
                (bonos_limites_comisiones['business_model'] == comisiones_mes_2024.iloc[i]['business_model'])
                & (bonos_limites_comisiones['is_gerente'] == True)
                & (bonos_limites_comisiones['lim_inf'] <= comisiones_mes_2024.iloc[i]['utilidad_$_after_regalos'])
                & (bonos_limites_comisiones['lim_sup'] >= comisiones_mes_2024.iloc[i]['utilidad_$_after_regalos'])
            ]

        else:
            
            bonos_limites_comisiones_line = bonos_limites_comisiones.loc[
                (bonos_limites_comisiones['business_model'] == comisiones_mes_2024.iloc[i]['business_model'])
                & (bonos_limites_comisiones['is_gerente'] == False)
                & (bonos_limites_comisiones['lim_inf'] <= comisiones_mes_2024.iloc[i]['utilidad_$_after_regalos'])
                & (bonos_limites_comisiones['lim_sup'] >= comisiones_mes_2024.iloc[i]['utilidad_$_after_regalos'])
            ]

            if bonos_limites_comisiones_line.empty:
                bonos_limites_comisiones_line = pd.DataFrame([[pd.NA, pd.NA]], columns = [ '%', 'bono'])

        comisiones_mes_2024.loc[i, ['comision_%']] = bonos_limites_comisiones_line[['%']].iloc[0].values
        comisiones_mes_2024.loc[i, ['comision_$']] = comisiones_mes_2024.loc[i, ['comision_%']].iloc[0] * comisiones_mes_2024.loc[i, ['utilidad_$_after_regalos']].iloc[0]
        comisiones_mes_2024.loc[i, ['bono_venta_$']] = bonos_limites_comisiones_line[['bono']].iloc[0].values


    return comisiones_mes_2024

In [129]:
def comision_cobranza_2024_func(cobranza_año:pd.DataFrame, datos_com_2023:pd.DataFrame, comisiones_2024:pd.DataFrame,  gerentes_list:pd.DataFrame) -> pd.DataFrame:
    
    cobranza_año.loc[:, ['fact_año']] = cobranza_año['fact_doc_date'].dt.year
    cobranza_año.loc[:, ['fact_mes']] = cobranza_año['fact_doc_date'].dt.month
    cobranza_año.loc[:, ['mes']] = cobranza_año['pay_date'].dt.month

    comision_cobranza_vend = cobranza_año.loc[
                                (cobranza_año['paga_comision'] == True)
                                & (~cobranza_año['salesperson_name'].isin(gerentes_list['salesperson_name']))
                                ,
                                [
                                    'salesperson_id',
                                    'salesperson_name',
                                    'mes',
                                    'pay_amount',
                                    'fact_año',
                                    'fact_mes'
                                ]
                            ].groupby(
                                ['salesperson_id', 
                                'salesperson_name', 
                                'mes', 
                                'fact_año', 
                                'fact_mes'
                                ]).sum().reset_index()


    comision_cobranza_ger = cobranza_año.loc[
                                (cobranza_año['paga_comision'] == True)
                                ,
                                [
                                    'sale_team_description',
                                    'mes',
                                    'pay_amount',
                                    'fact_año',
                                    'fact_mes'
                                ]
                            ].groupby(
                                ['sale_team_description', 
                                'mes', 
                                'fact_año', 
                                'fact_mes'
                                ]).sum().reset_index()


    comision_cobranza_ger.loc[
            comision_cobranza_ger['sale_team_description'].isin(['A1 CE', 'A2 CE'])
            ,
            ['salesperson_id', 'salesperson_name']
        ] = gerentes_list.loc[
                    gerentes_list['salesperson_name'] == 'Carlos Manuel Guzmán Rios'
                , 
                ['salesperson_id', 'salesperson_name']
            ].values

    comision_cobranza_ger.loc[
            comision_cobranza_ger['sale_team_description'] == 'A1 Piso'
            ,
            ['salesperson_id', 'salesperson_name']
        ] = gerentes_list.loc[
                    gerentes_list['salesperson_name'] == 'Mariana Araceli Carvajal Flores'
                , 
                ['salesperson_id', 'salesperson_name']
            ].values


    comision_cobranza_ger.loc[
            comision_cobranza_ger['sale_team_description'] == 'A2 Piso'
            ,
            ['salesperson_id', 'salesperson_name']
        ] = gerentes_list.loc[
                    gerentes_list['salesperson_name'] == 'Rosario Martinez Zarate'
                , 
                ['salesperson_id', 'salesperson_name']
            ].values

    comision_cobranza_ger['salesperson_id'] = comision_cobranza_ger['salesperson_id'].astype('Int64')


    comision_cobranza = pd.concat([comision_cobranza_vend, comision_cobranza_ger[comision_cobranza_vend.columns]], ignore_index=True)


    for i in range(len(comision_cobranza)):

        año = comision_cobranza.iloc[i]['fact_año']

        if año == 2022 or año == 2023:
            venta_comision_line = datos_com_2023.loc[
                                        (datos_com_2023['salesperson_id'] == comision_cobranza.iloc[i]['salesperson_id'])
                                        & (datos_com_2023['año'] == comision_cobranza.iloc[i]['fact_año'])
                                        & (datos_com_2023['mes'] == comision_cobranza.iloc[i]['fact_mes'])
                                    ]
        
        else:
            venta_comision_line = comisiones_2024.loc[
                                        (comisiones_2024['salesperson_id'] == comision_cobranza.iloc[i]['salesperson_id'])
                                        & (comisiones_2024['mes'] == comision_cobranza.iloc[i]['fact_mes'])
                                    ]
        
        if not venta_comision_line.empty:
            comision_cobranza.loc[i, ['facturacion_c/iva']] = venta_comision_line[['facturacion']].iloc[0].values * 1.16
            comision_cobranza.loc[i, ['cobranza_%']] = (comision_cobranza.loc[i, ['pay_amount']].iloc[0] / comision_cobranza.loc[i, ['facturacion_c/iva']].iloc[0]) * 100
            comision_cobranza.loc[i, ['comision_mes']] = venta_comision_line[['comision_$']].iloc[0].values
            comision_cobranza.loc[i, ['comision_cobranza_$']] = comision_cobranza.loc[i, ['cobranza_%']].iloc[0] / 100 * comision_cobranza.loc[i, ['comision_mes']].iloc[0]

    return comision_cobranza

In [140]:
def comision_pagos_2024_func(comisiones_2024:pd.DataFrame, comision_cobranza_2024:pd.DataFrame) -> pd.DataFrame:

    comision_pagos = comision_cobranza_2024[[
                                'salesperson_id', 
                                'salesperson_name', 
                                'mes',
                                'comision_cobranza_$'
                            ]].groupby([
                                'salesperson_id', 
                                'salesperson_name', 
                                'mes', 
                            ]).sum().reset_index().merge(
                                comisiones_2024[[
                                    'salesperson_id', 
                                    'Sale team description',
                                    'mes',
									'facturacion',
                                    'utilidad',
                                    'costo_regalo_$',
                                    'utilidad_$_after_regalos',
                                    'comision_%',
                                    'comision_$',
                                    'bono_venta_$',
                                ]],
                                how='left',
                                on=['salesperson_id', 'mes']
                            )

    comision_pagos.loc[
            ~comision_pagos[ 'bono_venta_$'].isna()
            ,
            'total_comision'
        ] = (comision_pagos['comision_cobranza_$'] + comision_pagos['bono_venta_$']) * 0.88

    comision_pagos.loc[
        comision_pagos[ 'bono_venta_$'].isna()
            ,
            'total_comision'
        ] = comision_pagos['comision_cobranza_$'] * 0.88


    return comision_pagos

In [141]:
gerentes_list = gerentes_list_func()

comisiones_1_2024 = comisiones_mes_2024_func(1, costo_ventas_list, gerentes_list, bonos_limites_comisiones, clientes_regalos)
comisiones_2_2024 = comisiones_mes_2024_func(2, costo_ventas_list, gerentes_list, bonos_limites_comisiones, clientes_regalos)
comisiones_3_2024 = comisiones_mes_2024_func(3, costo_ventas_list, gerentes_list, bonos_limites_comisiones, clientes_regalos)
comisiones_4_2024 = comisiones_mes_2024_func(4, costo_ventas_list, gerentes_list, bonos_limites_comisiones, clientes_regalos)
comisiones_5_2024 = comisiones_mes_2024_func(5, costo_ventas_list, gerentes_list, bonos_limites_comisiones, clientes_regalos)
comisiones_6_2024 = comisiones_mes_2024_func(6, costo_ventas_list, gerentes_list, bonos_limites_comisiones, clientes_regalos)
comisiones_7_2024 = comisiones_mes_2024_func(7, costo_ventas_list, gerentes_list, bonos_limites_comisiones, clientes_regalos)
comisiones_8_2024 = comisiones_mes_2024_func(8, costo_ventas_list, gerentes_list, bonos_limites_comisiones, clientes_regalos)
comisiones_9_2024 = comisiones_mes_2024_func(9, costo_ventas_list, gerentes_list, bonos_limites_comisiones, clientes_regalos)
comisiones_10_2024 = comisiones_mes_2024_func(10, costo_ventas_list, gerentes_list, bonos_limites_comisiones, clientes_regalos)
comisiones_11_2024 = comisiones_mes_2024_func(11, costo_ventas_list, gerentes_list, bonos_limites_comisiones, clientes_regalos)
# comisiones_12_2024 = comisiones_mes_2024_func(12, costo_ventas_list, gerentes_list, bonos_limites_comisiones, clientes_regalos)

comisiones_2024 = pd.concat([
                        comisiones_1_2024,
                        comisiones_2_2024,
                        comisiones_3_2024,
                        comisiones_4_2024,
                        comisiones_5_2024,
                        comisiones_6_2024,
                        comisiones_7_2024,
                        comisiones_8_2024,
                        comisiones_9_2024,
                        comisiones_10_2024,
                        comisiones_11_2024,
                        # comisiones_12_2024,
                    ])

comision_cobranza_2024 = comision_cobranza_2024_func(cobranza_año, datos_com_2023, comisiones_2024, gerentes_list)

comision_pagos_2024 = comision_pagos_2024_func(comisiones_2024, comision_cobranza_2024)

In [ ]:
from IPython.display import display
    
id_vend = 209

display(comision_pagos_2024.loc[
        	comision_pagos_2024['salesperson_id'] == id_vend
            ,
				[
				'salesperson_id',
				'salesperson_name',
				'Sale team description',
				'mes',
				'facturacion',
				'utilidad',
				'comision_%',
				'comision_$',
				'comision_cobranza_$',
				'bono_venta_$',
				'total_comision',
				]
			]
    )

display(comision_pagos_2024.loc[
        comision_pagos_2024['salesperson_id'] == id_vend
        ,
        ['salesperson_name', 'total_comision']
    ]
        .groupby('salesperson_name')
        .sum()
    )

In [ ]:
mes = 11

comision_pagos_2024[comision_pagos_2024['mes'] == mes]

# comision_pagos_2024[comision_pagos_2024['mes'] == mes].to_excel(str(Path.home().joinpath(f'desktop/comisiones_{mes}_2024.xlsx')))
# print('Se generó el archivo excel con éxito')

In [134]:
# with engine.connect() as conn, conn.begin():  
#     comisiones_2024.to_sql(name = 'comisiones_2024', index= False, con = conn, if_exists = 'replace')
#     comision_cobranza_2024.to_sql(name = 'comision_cobranza_2024', index= False, con = conn, if_exists = 'replace')

# engine.dispose()

# print('Todas las tablas se ingresaron con éxito')